In [44]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
import smtplib
import ssl
import logging
import datetime
import psycopg2
from configparser import ConfigParser


def config(filename, section):
    parser = ConfigParser()
    parser.read(filename)

    con = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            con[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return con


def articles_urls():
    response = requests.get('https://mfn.se/all/s')
    content = BeautifulSoup(response.content, "html.parser")
    article_urls = []

    for article in content.find_all(
            class_='short-item-wrapper grid-u-1 grid-u-md-1-2 grid-u-lg-1-3 grid-u-xl-1-4 removable-grid'):
        url = article.find(class_='short-item compressible').get('onclick').replace("goToNewsItem(event, '",
                                                                                    '').replace("')", '')
        article_urls.append(f'https://mfn.se{url}')

    return article_urls


def article_content(article_url):
    response = requests.get(article_url)
    content = BeautifulSoup(response.content, "html.parser")

    company = str(content.find(class_='tray company').getText().replace('\n', ''))
    dt = datetime.datetime.strptime(
        content.find(class_='full-item').find(class_='publish-date').getText().replace('\n', '').strip(),
        '%Y-%m-%d %H:%M:%S')
    subject = content.find(class_='').find(class_='title').getText().replace('\n', '')
    text = content.find(class_='full-item').find(class_='mfn-preamble').getText().replace('\n', '')

    return {'company': company, 'dt': dt, 'subject': subject, 'text': text, 'url': article_url}


def articles_info():
    response = requests.get('https://mfn.se/all/s')
    content = BeautifulSoup(response.content, "html.parser").find_all(
        class_='short-item-wrapper grid-u-1 grid-u-md-1-2 grid-u-lg-1-3 grid-u-xl-1-4 removable-grid')
    articles = []

    for article in content[:4]:
        url = article.find(class_='short-item compressible').get('onclick').replace("goToNewsItem(event, '",
                                                                                    '').replace("')", '')
        company = content[0].find(class_='compressed-author').getText().replace('\n', '')
        d = content[0].find(class_='compressed-date').getText().replace('\n', '')
        t = content[0].find(class_='compressed-time').getText().replace('\n', '')
        dt = datetime.datetime.strptime('{} {}'.format(d, t), '%Y-%m-%d %H:%M:%S')
        url = f'https://mfn.se{url}'

        articles.append({'company': company, 'url': url, 'dt': dt})

    return articles


def stock_price_from_news(content):
    patterns = ['\$(\d*[.,]?\d*) per Common Share', 'price/share, EUR</td><td>(\d*[.,]?\d*)</td><td>',
                'Keskihinta/ osake</td><td>(\d*[.,]?\d*)</td><td>', 'at an average price of NOK (\d*[.,]?\d*)',
                'at an average price per share of NOK (\d*[.,]?\d*)', 'shares at NOK (\d*[.,]?\d*) pr share',
                '(\d*[.,]?\d*) per share', '(\d*[.,]?\d*) per share', '[Pp]?rice per share (\d*[.,]?\d*)(\d*[.,]?\d*)']
    pattern_matches = []

    for pattern in patterns:
        pattern_matches += re.findall(pattern, content.replace('\n', ' '))

    prices = []
    for price in pattern_matches:
        try:
            price = float(price.replace(',', '.'))

            if price > 0:
                prices.append(price)
        except:
            pass

    return prices[0] if prices else False


def company_page(company):
    headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'}
    response = requests.get(f'https://www.investing.com/search/?q={urllib.parse.quote(company)}', headers=headers)
    content = BeautifulSoup(response.content, "html.parser")
    url = 'https://www.investing.com{}'.format(content.find(class_='js-inner-all-results-quote-item row').get('href'))

    return url


def company_stock_price(company):
    url = company_page(company)
    headers = {
        "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'}
    response = requests.get(url, headers=headers)
    content = BeautifulSoup(response.content, "html.parser")
    price = content.find(id='quotes_summary_current_data').find(class_='inlineblock').find(id='last_last').getText()

    return float(price)


def send_email(subject, text, receiver_email):
    sender_email = config('config.ini', 'sender')['email']
    password = config('config.ini', 'sender')['password']
    message = '''\
    Subject: {}

    {}'''.format(subject, text)

    context = ssl.create_default_context()
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.ehlo()
        server.starttls(context=context)
        server.ehlo()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)


def get_article_price(company, dt):   # @TODO
    return 1


def get_current_price(company):  # @TODO
    return 2


insert_tpl = '''INSERT INTO public.alerts 
(current_dt, news_dt, stock_name, news_subject, url, news_text, news_dt_price, current_dt_price) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'''  # TODO sure about public.alerts??


def main():
    conn = psycopg2.connect(**config('config.ini', 'database'))
    receiver_email = config('config.ini', 'receiver')['email']
    articles = articles_urls()

    for article_url in articles:
        article = article_content(article_url)
        article_price = get_article_price(article['company'], article['dt'])
        current_price = get_current_price(article['company'])
        ratio = article_price / current_price

        logging.INFO('''Price for {} at moment of news is {}. Current stock price {}. Ratio = {}
            Article {}'''.format(article['company'], article_price, current_price, ratio, article['url']))

        if ratio > 1.03:
            text = '''Price for {} at moment of news is {}. Current stock price {}.
            Article {}'''.format(article['company'], article_price, current_price, article['url'])

            with conn.cursor() as cursor:
                send_email(article['company'], text, receiver_email)
                logging.INFO('Email sent')

                cursor.execute(insert_tpl, (
                    datetime.datetime.now(), article['dt'], article['company'], article['subject'], article['url'],
                    article['text'], article_price, current_price))
                logging.INFO('Row inserted')

    conn.close()

# @TODO add requirements file
# @TODO ask if send multiple alerts






In [ ]:

main()

In [43]:

article_url = 'https://mfn.se/cis/a/brighter/brighter-ab-publ-the-extraordinary-general-meeting-will-take-a-stand-on-a-new-proposal-for-the-chairman-of-the-board-96390109'

#     return {'company': company, 'content': content}

"On July 28, Brighter's Nomination Committee presented a proposal for a new Board of Directors and Chairman of the Board of Brighter ahead of the Extraordinary General Meeting tomorrow, July 31 at 10.00. Shortly before the Extraordinary General Meeting, the former candidate for Chariman of the Board, Håkan Nyberg, has chosen to decline his nomination due to personal reasons. Therefore, a new proposal for the Chairman of the Board will be presented ahead of the Extraordinary General Meeting."

In [29]:
print(articles_urls())

['https://mfn.se/cis/a/brighter/brighter-ab-publ-the-extraordinary-general-meeting-will-take-a-stand-on-a-new-proposal-for-the-chairman-of-the-board-96390109', 'https://mfn.se/cis/a/brighter/brighter-ab-publ-extrastamman-kommer-att-fa-ta-stallning-till-nytt-forslag-pa-styrelseordforande-c14bc92a', 'https://mfn.se/one/a/air-france-klm-sa/air-france-klm-second-quarter-2020-results-16b1d18b', 'https://mfn.se/one/a/air-france-klm-sa/resultat-du-deuxieme-trimestre-2020-air-france-klm-ae80fd97', 'https://mfn.se/one/a/rovio-entertainment/rovio-entertainment-oyj-omien-osakkeiden-hankinta-30-07-2020-53973657', 'https://mfn.se/one/a/rovio-entertainment/rovio-entertainment-corporation-repurchase-of-own-shares-on-30-july-2020-96fe23a1', 'https://mfn.se/one/a/investors-house/investors-house-oyj-omien-osakkeiden-hankinta-30-7-2020-679ae59e', 'https://mfn.se/one/a/vincit/vincit-oyj-omien-osakkeiden-hankinta-30-7-2020-91d9205a', 'https://mfn.se/cis/a/analyst-group/analyst-group-aktieanalys-pa-obducat-

In [23]:
article = {'url': 'sss'}
conn = psycopg2.connect(**config('config.ini', 'database'))
with conn.cursor() as cursor:
    if not cursor.execute("SELECT * FROM public.alerts WHERE url = '{}'".format(article['url'])):
        print(11)
    print(cursor.fetchall())

11
[]


In [25]:
datetime.datetime.now()

datetime.datetime(2020, 7, 30, 20, 2, 40, 53935)

In [16]:
response = requests.get('https://mfn.se/all/s')
content = BeautifulSoup(response.content, "html.parser").find_all(
        class_='short-item-wrapper grid-u-1 grid-u-md-1-2 grid-u-lg-1-3 grid-u-xl-1-4 removable-grid')
articles = []

for article in content[:4]:
    url = article.find(class_='short-item compressible').get('onclick').replace("goToNewsItem(event, '",
                                                                                '').replace("')", '')
    company = content[0].find(class_='compressed-author').getText().replace('\n', '')
    d = content[0].find(class_='compressed-date').getText().replace('\n', '')
    t = content[0].find(class_='compressed-time').getText().replace('\n', '')
    dt = datetime.datetime.strptime('{} {}'.format(d, t), '%Y-%m-%d %H:%M:%S')
    url = article_urls.append(f'https://mfn.se{url}')
    
    articles.append({'company': company, 'url': url, 'dt': dt})


In [17]:
articles

[{'company': 'Betsson',
  'url': None,
  'dt': datetime.datetime(2020, 7, 30, 13, 0)},
 {'company': 'Betsson',
  'url': None,
  'dt': datetime.datetime(2020, 7, 30, 13, 0)},
 {'company': 'Betsson',
  'url': None,
  'dt': datetime.datetime(2020, 7, 30, 13, 0)},
 {'company': 'Betsson',
  'url': None,
  'dt': datetime.datetime(2020, 7, 30, 13, 0)}]

In [9]:
dt = content[0].find(class_='compressed-date').getText().replace('\n', '')

'2020-07-30'

In [10]:
t = content[0].find(class_='compressed-time').getText().replace('\n', '')

'13:00:00'

In [44]:

articles = ['https://mfn.se/one/a/rovio-entertainment/rovio-entertainment-corporation-repurchase-of-own-shares-on-29-july-2020-70d692d3',
           'https://mfn.se/one/a/vincit/vincit-oyj-omien-osakkeiden-hankinta-29-7-2020-41b4cd15',
           'https://mfn.se/ob/a/bergenbio/bergenbio-asa-primary-insider-notification-282b2519',
           'https://mfn.se/ob/a/atlantic-sapphire/atlantic-sapphire-asa-primary-insider-notification-fea795c7',
           'https://mfn.se/ob/a/telenor/tel-trades-by-primary-insider-8ebc6150',
           'https://mfn.se/one/a/orphazyme/reporting-of-transactions-in-orphazymes-shares-made-by-persons-discharging-managerial-responsibilities-20dbe2ea',
           'https://mfn.se/cis/a/nordic-waterproofing-holding/reporting-of-transactions-made-by-persons-discharging-managerial-responsibilities-and-persons-closely-associated-with-them-in-nordic-waterproofing-holding-a-s-shares-318b199a']


for article_url in articles:
    response = article_content(article_url)
    price = stock_price_from_news(response['content'])
    if price:
        print('Price for {} is {}'.format(response['company'], price))
    else:
        print('No price for {}'.format(response['company']))
        
        
        

Price of Rovio Entertainment is 5.661667
Price of Vincit is 4.5715
Price of BerGenBio is 31.4422
Price of Atlantic Sapphire is 113.0
Price of Telenor is 148.0
No price for Orphazyme
Price of Nordic Waterproofing Holding is 120.7052


In [21]:
re.findall('Price per share (\d*[.,]?\d*)', content.replace('\n', ' '))

['120.7052', '120.7052']

In [15]:
a = 'b'
print(a)

a.replace('b', 'AAAAA')

print(a)

b
b


In [22]:
pattern = 
content = article_content('https://mfn.se/cis/a/nordic-waterproofing-holding/reporting-of-transactions-made-by-persons-discharging-managerial-responsibilities-and-persons-closely-associated-with-them-in-nordic-waterproofing-holding-a-s-shares-318b199a')['content']
content


'<div class="full-item" id="20dbe2ea-a40e-41ea-994f-bb43524f49ea">\n<div class="title">\n<a href="/one/a/orphazyme/reporting-of-transactions-in-orphazymes-shares-made-by-persons-discharging-managerial-responsibilities-20dbe2ea">Reporting of transactions in Orphazyme’s shares made by persons discharging managerial responsibilities</a>\n</div>\n<div class="publish-date">\n      2020-07-29 14:51:47\n\n</div>\n<div class="content s-one">\n<div class="mfn-preamble"><strong></strong></div><div><p><strong>Orphazyme A/S<br/>Company announcement\xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0</strong><br/>No. 43/2020<strong>\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\x

In [ ]:
# price/share   https://mfn.se/one/a/rovio-entertainment/rovio-entertainment-corporation-repurchase-of-own-shares-on-29-july-2020-70d692d3

# Keskihinta/ osake   https://mfn.se/one/a/vincit/vincit-oyj-omien-osakkeiden-hankinta-29-7-2020-41b4cd15

# at an average price of NOK 31.442  https://mfn.se/ob/a/bergenbio/bergenbio-asa-primary-insider-notification-282b2519
# at an average price per share of NOK 113.0 https://mfn.se/ob/a/atlantic-sapphire/atlantic-sapphire-asa-primary-insider-notification-fea795c7
# shares at NOK 148.00 pr share. https://mfn.se/ob/a/telenor/tel-trades-by-primary-insider-8ebc6150

# Price per share https://mfn.se/one/a/orphazyme/reporting-of-transactions-in-orphazymes-shares-made-by-persons-discharging-managerial-responsibilities-20dbe2ea 
#   per share https://mfn.se/cis/a/nordic-waterproofing-holding/reporting-of-transactions-made-by-persons-discharging-managerial-responsibilities-and-persons-closely-associated-with-them-in-nordic-waterproofing-holding-a-s-shares-318b199a


In [135]:
company_stock_price('https://www.investing.com/equities/ngex-resources-inc')

0.73

In [95]:

pattern_matches = re.findall('\$(\d*.?\d*) per Common Share', '$23145 per Common Share fsffs $0.2333 per Common Share ')
pattern_matches

# text    = "city.state.domain.com"
# pattern = "^([^\.]+).([^\.]+).([^\.]+).([^\.]+)$"
# match   = re.match(pattern, text)
# match.groups()


['23145', '0.2333']

In [121]:
headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'}
response = requests.get('https://www.investing.com/equities/ngex-resources-inc', headers=headers)
content = BeautifulSoup(response.content, "html.parser")
content

 <!DOCTYPE HTML>

<html class="com" dir="ltr" geo="RU" itemscope="" itemtype="http://schema.org/Corporation" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:schema="http://schema.org/">
<head>
<script>
        (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
            (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
            m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
        })(window,document,'script','//www.google-analytics.com/analytics.js','ga');
    </script>
<script type="text/javascript">
        window.helpers = {
            'getCookie': function (k){var v="; "+document.cookie;var p=v.split("; "+k+"=");if(p.length===2)return p.pop().split(";").shift();else return null},
            'setCookie': function (k,v,d){var e='';if(typeof d!=='undefined'){var n=new Date;n.setTime(n.getTime()+864e5*d);e+=';expires="';e+=n.toGMTString();e+='"';}document.cookie=k+'='+v+';

In [129]:
content.find(id='quotes_summary_current_data').find(class_='inlineblock').find(id='last_last').getText()

'0.73'

In [107]:
content.find(class_='tray company').getText().replace('\n', '')

'Josemaria Resources'

In [ ]:
Josemaria Resources

In [114]:
company = 'Josemaria Resources'
headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'}
response = requests.get(f'https://www.investing.com/search/?q={urllib.parse.quote(company)}', headers=headers)
content = BeautifulSoup(response.content, "html.parser")
url = 'https://www.investing.com/' + content.find(class_='js-inner-all-results-quote-item row').get('href')



'/equities/ngex-resources-inc'

In [76]:

content = article_content('https://mfn.se/cis/a/josemaria-resources/josemaria-resources-inc-announces-c-25-million-financing-c92d8849')


In [38]:
content

<div class="full-item" id="c92d8849-9916-4373-a384-00765737028d">
<div class="title">
<a href="/cis/a/josemaria-resources/josemaria-resources-inc-announces-c-25-million-financing-c92d8849">Josemaria Resources Inc. Announces C$25 Million Financing</a>
</div>
<div class="publish-date">
      2020-07-28 22:10:59

</div>
<div class="content s-cis">
<div class="mfn-preamble"><strong><p>NOT FOR DISTRIBUTION TO U.S. NEWSWIRE SERVICES OR FOR RELEASE, PUBLICATION, DISTRIBUTION OR DISSEMINATION DIRECTLY OR INDIRECTLY, IN WHOLE OR IN PART, IN OR INTO THE UNITED STATES.</p></strong></div><p>VANCOUVER, July 28, 2020 /CNW/ - Josemaria Resources Inc. (TSX: JOSE) (OMX: JOSE) ("Josemaria Resources," "Josemaria," or the "Company") has announced today that it has entered into an agreement with a syndicate of underwriters led by BMO Capital Markets (the "Underwriters"), under which the Underwriters have agreed to buy on bought deal basis 15,000,000 common shares (the "Common Shares"), at a price of C$0.67

In [29]:
content.find_all(class_='full-item')

[<div class="full-item" id="c92d8849-9916-4373-a384-00765737028d">
 <div class="title">
 <a href="/cis/a/josemaria-resources/josemaria-resources-inc-announces-c-25-million-financing-c92d8849">Josemaria Resources Inc. Announces C$25 Million Financing</a>
 </div>
 <div class="publish-date">
       2020-07-28 22:10:59
 
 </div>
 <div class="content s-cis">
 <div class="mfn-preamble"><strong><p>NOT FOR DISTRIBUTION TO U.S. NEWSWIRE SERVICES OR FOR RELEASE, PUBLICATION, DISTRIBUTION OR DISSEMINATION DIRECTLY OR INDIRECTLY, IN WHOLE OR IN PART, IN OR INTO THE UNITED STATES.</p></strong></div><p>VANCOUVER, July 28, 2020 /CNW/ - Josemaria Resources Inc. (TSX: JOSE) (OMX: JOSE) ("Josemaria Resources," "Josemaria," or the "Company") has announced today that it has entered into an agreement with a syndicate of underwriters led by BMO Capital Markets (the "Underwriters"), under which the Underwriters have agreed to buy on bought deal basis 15,000,000 common shares (the "Common Shares"), at a price

In [21]:

for i in content.find_all(class_='short-item-wrapper grid-u-1 grid-u-md-1-2 grid-u-lg-1-3 grid-u-xl-1-4 removable-grid'):
    print(i.find(class_='short-item compressible').get('onclick').replace("goToNewsItem(event, '", '').replace("')", ''))
    
    

/cis/a/josemaria-resources/josemaria-resources-inc-announces-c-25-million-financing-c92d8849
/one/a/ntg-nordic-transport-group/transactions-in-connection-with-share-buyback-program-8485d206
/one/a/mondelez-international/mondelez-international-reports-q2-2020-results-1eb63878
/cis/a/nordic-waterproofing-holding/reporting-of-transactions-made-by-persons-discharging-managerial-responsibilities-and-persons-closely-associated-with-them-in-nordic-waterproofing-holding-a-s-shares-4c5124f2
/cis/a/nordic-waterproofing-holding/rapportering-av-transaktioner-i-nordic-waterproofing-holding-a-s-aktier-gjorda-av-ledande-befattningshavare-och-deras-narstaende-fe47f32f
/one/a/orion/orions-phase-3-refals-trial-evaluating-the-efficacy-of-oral-levosimendan-in-treatment-of-als-patients-did-not-reach-its-pre-specified-endpoints-1c00f2e4
/one/a/orion/suun-kautta-annosteltavan-levosimendaanin-tehoa-als-potilaiden-hoidossa-arvioiva-orionin-faasi-3-refals-tutkimus-ei-saavuttanut-asetettuja-tavoitteitaan-dc5c09d